# K-means clustering.

The aim of this interactive demo is to show the sensitivity of the K-means clustering algorithm to the initial centroid locations. We start with unlabelled data with two features. Visually, we observe this can be classified into three categories. Setting K=3, we proceed to compute the clusters. Adjust the sliders below and see how the clustering evolves. You can run one epoch at a time to see slowly how the algorithm proceeds or run 10 epochs at once. The centroids are denoted with the coloured crosses. The colours correspond to the different classes. 

## Things to try. 
The Algorithm is relatively robust to broad ranges of initial conditions (for this particular dataset at least). But what happens if you set the initial centroids all at (or close to) -10, -10? Have a think why this is, and what you could do to prevent it.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import Markdown
import functools

In [2]:
from sklearn import datasets
blobs = datasets.make_blobs(n_samples=300, cluster_std=1.5, random_state=12345)
X=blobs[0]

In [3]:
def assign(mu1, mu2, mu3, X):
    Y = np.zeros(len(X))
    for i in range(len(X)):
        dist1 = (X[i][0] - mu1[0])**2 + (X[i][1] - mu1[1])**2
        dist2 = (X[i][0] - mu2[0])**2 + (X[i][1] - mu2[1])**2
        dist3 = (X[i][0] - mu3[0])**2 + (X[i][1] - mu3[1])**2

        if( dist1 <= dist2 and dist1 <= dist3 ):
            Y[i] = 1
        elif( dist2 <= dist1 and dist2 <= dist3 ):
            Y[i] = 2
        else:
            Y[i] = 3  
    return Y    

In [4]:
def centroids(X, Y):
    mu1l = np.array([0,0])
    mu2l = np.array([0,0])
    mu3l = np.array([0,0])

    n1=0
    n2=0
    n3=0
    
    for i in range(len(Y)):
        if(Y[i] == 1 ):
            mu1l = mu1l + X[i]
            n1 += 1
        elif(Y[i] == 2 ):
            mu2l = mu2l + X[i]
            n2 += 1
        else:
            mu3l = mu3l + X[i]
            n3 += 1

    if(n1 != 0 ):
        mu1l = mu1l/(n1+1e-10)
    else:
        mu1l = mu1

    if(n2 != 0 ):
        mu2l = mu2l/(n2+1e-10)
    else:
        mu2l = mu2

    if(n3 != 0 ):
        mu3l = mu3l/(n3+1e-10)
    else:
        mu3l = mu3
        
    return mu1l, mu2l, mu3l

In [5]:
colors = np.array([x for x in 'bgrcmykbgrcmykbgrcmykbgrcmyk'])
colors = np.hstack([colors] * 20)

In [7]:
try:
    plt.style.use('seaborn')
except:
    pass


initial_mu1_x1 = widgets.FloatSlider(2, min=-10, max=10.0, description=r'Initial $\mu_1 x$:', continuous_update = False)
initial_mu2_x1 = widgets.FloatSlider(3, min=-10, max=10.0, description=r'Initial $\mu_2 x$:', continuous_update = False)
initial_mu3_x1 = widgets.FloatSlider(-3, min=-10, max=10.0, description=r'Initial $\mu_3 x$:', continuous_update = False)

initial_mu1_y1 = widgets.FloatSlider(2, min=-10, max=10.0, orientation='vertical', description=r'Initial $\mu_1 y$:', continuous_update = False)
initial_mu2_y1 = widgets.FloatSlider(3, min=-10, max=10.0, orientation='vertical', description=r'Initial $\mu_2 y$:', continuous_update = False)
initial_mu3_y1 = widgets.FloatSlider(-3, min=-10, max=10.0, orientation='vertical', description=r'Initial $\mu_3 y$:', continuous_update = False)

left_box1 = widgets.VBox([initial_mu1_x1, initial_mu2_x1, initial_mu3_x1 ])
right_box1 = widgets.HBox([initial_mu1_y1, initial_mu2_y1, initial_mu3_y1])
sliders1 = widgets.HBox([left_box1, right_box1])

sub1_1 = widgets.Button(description='1 epoch')
sub10_1 = widgets.Button(description='10 epochs')
out1 = widgets.Output()

#display(initial_mu1_x, initial_mu2_x, initial_mu3_x)
#display(initial_mu1_y, initial_mu2_y, initial_mu3_y)
display(sliders1)
display(sub1_1, sub10_1, out1)

Y = np.zeros(len(X), dtype=int)

mu1_1 = np.array([initial_mu1_x1.value, initial_mu1_y1.value])
mu2_1 = np.array([initial_mu2_x1.value, initial_mu2_y1.value])
mu3_1 = np.array([initial_mu3_x1.value, initial_mu3_y1.value])

def train1(epochs):
    out1.clear_output(wait=True)
    with out1:
        global mu1_1, mu2_1, mu3_1
        for _ in range(int(epochs)):
            Y = assign(mu1_1, mu2_1, mu3_1, X)
            mu1_1, mu2_1, mu3_1 = centroids(X, Y)
        plt.scatter(X[:,0], X[:,1], color=colors[Y.astype(int)])

        plt.scatter(mu1_1[0], mu1_1[1], edgecolor = 'k', color=colors[1], s = 200, marker='X')
        plt.scatter(mu2_1[0], mu2_1[1], edgecolor = 'k', color=colors[2], s = 200, marker='X')
        plt.scatter(mu3_1[0], mu3_1[1], edgecolor = 'k', color=colors[3], s = 200, marker='X')
        plt.xlabel(r'$x_1$')
        plt.ylabel(r'$x_2$')
            
        plt.show()



def reset_Y1(b=None):
    out1.clear_output(wait=True)
    with out1:
        global mu1_1, mu2_1, mu3_1
        mu1_1 = np.array([initial_mu1_x1.value, initial_mu1_y1.value])
        mu2_1 = np.array([initial_mu2_x1.value, initial_mu2_y1.value])
        mu3_1 = np.array([initial_mu3_x1.value, initial_mu3_y1.value])
        Y = np.zeros(len(X), dtype=int)
        plt.scatter(X[:,0], X[:,1], color=colors[Y.astype(int)])
        plt.scatter(mu1_1[0], mu1_1[1], edgecolor = 'k', color=colors[1], s = 200, marker='X')
        plt.scatter(mu2_1[0], mu2_1[1], edgecolor = 'k', color=colors[2], s = 200, marker='X')
        plt.scatter(mu3_1[0], mu3_1[1], edgecolor = 'k', color=colors[3], s = 200, marker='X')
        plt.xlabel(r'$x_1$')
        plt.ylabel(r'$x_2$')
        
        plt.show()

def train1_1(b):
    train1(1)

def train10_1(b):
    train1(10)

reset_Y1()

initial_mu1_x1.observe(reset_Y1)
initial_mu2_x1.observe(reset_Y1)
initial_mu3_x1.observe(reset_Y1)
initial_mu1_y1.observe(reset_Y1)
initial_mu2_y1.observe(reset_Y1)
initial_mu3_y1.observe(reset_Y1)
sub1_1.on_click(train1_1)
sub10_1.on_click(train10_1)

Button(description='1 epoch', style=ButtonStyle())

Button(description='10 epochs', style=ButtonStyle())

Output()

# Data with outliers. 

In the next example, we have the same dataset but with the addition of a single outlier. See what happens if one of the initial centroids gets too close to the outlier. 

In [8]:
try:
    plt.style.use('seaborn')
except:
    pass


X2 = X
X2 = np.concatenate((X2, np.array([[13,13]])))

initial_mu1_x2 = widgets.FloatSlider(10, min=-10, max=10.0, description=r'Initial $\mu_1 x$:', continuous_update = False)
initial_mu2_x2 = widgets.FloatSlider(3, min=-10, max=10.0, description=r'Initial $\mu_2 x$:', continuous_update = False)
initial_mu3_x2 = widgets.FloatSlider(-3, min=-10, max=10.0, description=r'Initial $\mu_3 x$:', continuous_update = False)

initial_mu1_y2 = widgets.FloatSlider(10, min=-10, max=10.0, orientation='vertical', description=r'Initial $\mu_1 y$:', continuous_update = False)
initial_mu2_y2 = widgets.FloatSlider(3, min=-10, max=10.0, orientation='vertical', description=r'Initial $\mu_2 y$:', continuous_update = False)
initial_mu3_y2 = widgets.FloatSlider(-3, min=-10, max=10.0, orientation='vertical', description=r'Initial $\mu_3 y$:', continuous_update = False)

left_box2 = widgets.VBox([initial_mu1_x2, initial_mu2_x2, initial_mu3_x2 ])
right_box2 = widgets.HBox([initial_mu1_y2, initial_mu2_y2, initial_mu3_y2])
sliders2 = widgets.HBox([left_box2, right_box2])

sub1_2 = widgets.Button(description='1 epoch')
sub10_2 = widgets.Button(description='10 epochs')
out2 = widgets.Output()

#display(initial_mu1_x, initial_mu2_x, initial_mu3_x)
#display(initial_mu1_y, initial_mu2_y, initial_mu3_y)
display(sliders2)
display(sub1_2, sub10_2, out2)

Y2 = np.zeros(len(X2), dtype=int)

mu1_2 = np.array([initial_mu1_x2.value, initial_mu1_y2.value])
mu2_2 = np.array([initial_mu2_x2.value, initial_mu2_y2.value])
mu3_2 = np.array([initial_mu3_x2.value, initial_mu3_y2.value])

def train2(epochs):
    out2.clear_output(wait=True)
    with out2:
        global mu1_2, mu2_2, mu3_2
        for _ in range(int(epochs)):
            Y2 = assign(mu1_2, mu2_2, mu3_2, X2)
            mu1_2, mu2_2, mu3_2 = centroids(X2, Y2)
        plt.scatter(X2[:,0], X2[:,1], color=colors[Y2.astype(int)])

        plt.scatter(mu1_2[0], mu1_2[1], edgecolor = 'k', color=colors[1], s = 200, marker='X')
        plt.scatter(mu2_2[0], mu2_2[1], edgecolor = 'k', color=colors[2], s = 200, marker='X')
        plt.scatter(mu3_2[0], mu3_2[1], edgecolor = 'k', color=colors[3], s = 200, marker='X')
        plt.xlabel(r'$x_1$')
        plt.ylabel(r'$x_2$')
            
        plt.show()



def reset_Y2(b=None):
    out2.clear_output(wait=True)
    with out2:
        global mu1_2, mu2_2, mu3_2
        mu1_2 = np.array([initial_mu1_x2.value, initial_mu1_y2.value])
        mu2_2 = np.array([initial_mu2_x2.value, initial_mu2_y2.value])
        mu3_2 = np.array([initial_mu3_x2.value, initial_mu3_y2.value])
        Y2 = np.zeros(len(X2), dtype=int)
        plt.scatter(X2[:,0], X2[:,1], color=colors[Y2.astype(int)])
        plt.scatter(mu1_2[0], mu1_2[1], edgecolor = 'k', color=colors[1], s = 200, marker='X')
        plt.scatter(mu2_2[0], mu2_2[1], edgecolor = 'k', color=colors[2], s = 200, marker='X')
        plt.scatter(mu3_2[0], mu3_2[1], edgecolor = 'k', color=colors[3], s = 200, marker='X')
        plt.xlabel(r'$x_1$')
        plt.ylabel(r'$x_2$')
        
        plt.show()

def train1_2(b):
    train2(1)

def train10_2(b):
    train2(10)

reset_Y2()

initial_mu1_x2.observe(reset_Y2)
initial_mu2_x2.observe(reset_Y2)
initial_mu3_x2.observe(reset_Y2)
initial_mu1_y2.observe(reset_Y2)
initial_mu2_y2.observe(reset_Y2)
initial_mu3_y2.observe(reset_Y2)
sub1_2.on_click(train1_2)
sub10_2.on_click(train10_2)

Button(description='1 epoch', style=ButtonStyle())

Button(description='10 epochs', style=ButtonStyle())

Output()